In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

In [2]:
data_path = r'RSandUSBR_data.csv'

df = pd.read_csv(data_path)

In [3]:
df.head()

,Unnamed: 0,Date,Precipitation,Precipitation_(-1d),Precipitation_(-7d),Volumetric Soil moisture,2m_Temp,Discharge,Evaporation,Sat_Area,Real time storage
0,0,1/14/1990,0.0,0.000,0.000,0.005457,8.307883,622.0,232.819472,83.057795,1692000
1,1,4/4/1990,0.0,0.000,0.000,0.056118,14.296631,385.0,280.031540,81.264840,1639600
2,2,4/11/1990,0.0,0.000,0.000,0.022323,17.140956,36.0,661.206166,80.886614,1626800
3,3,4/20/1990,0.0,0.000,0.000,0.113441,13.940720,0.0,536.174092,80.579682,1613600
4,4,5/6/1990,0.0,0.319,1.164,0.114180,14.372926,523.0,563.486792,80.213382,1586200


In [4]:
#max_value = df['swv'].max()
#max_value



In [5]:
# @title GW

#from matplotlib import pyplot as plt
#df['GW'].plot(kind='hist', bins=20, title='GW')
#plt.gca().spines[['top', 'right',]].set_visible(False)

In [6]:
df.index = pd.to_datetime(df.Date)

In [7]:
df.head()

,Unnamed: 0,Date,Precipitation,Precipitation_(-1d),Precipitation_(-7d),Volumetric Soil moisture,2m_Temp,Discharge,Evaporation,Sat_Area,Real time storage
Date,,,,,,,,,,,
1990-01-14,0,1/14/1990,0.0,0.000,0.000,0.005457,8.307883,622.0,232.819472,83.057795,1692000
1990-04-04,1,4/4/1990,0.0,0.000,0.000,0.056118,14.296631,385.0,280.031540,81.264840,1639600
1990-04-11,2,4/11/1990,0.0,0.000,0.000,0.022323,17.140956,36.0,661.206166,80.886614,1626800
1990-04-20,3,4/20/1990,0.0,0.000,0.000,0.113441,13.940720,0.0,536.174092,80.579682,1613600
1990-05-06,4,5/6/1990,0.0,0.319,1.164,0.114180,14.372926,523.0,563.486792,80.213382,1586200


In [8]:
required_cols = ['Real time storage','Precipitation', 'Precipitation_(-1d)', 'Precipitation_(-7d)', 'Volumetric Soil moisture', '2m_Temp', 'Discharge', 'Evaporation', 'Sat_Area']
#required_cols = ['Real time storage','Sat_Area']
df = df[required_cols]
df.head()

,Real time storage,Precipitation,Precipitation_(-1d),Precipitation_(-7d),Volumetric Soil moisture,2m_Temp,Discharge,Evaporation,Sat_Area
Date,,,,,,,,,
1990-01-14,1692000,0.0,0.000,0.000,0.005457,8.307883,622.0,232.819472,83.057795
1990-04-04,1639600,0.0,0.000,0.000,0.056118,14.296631,385.0,280.031540,81.264840
1990-04-11,1626800,0.0,0.000,0.000,0.022323,17.140956,36.0,661.206166,80.886614
1990-04-20,1613600,0.0,0.000,0.000,0.113441,13.940720,0.0,536.174092,80.579682
1990-05-06,1586200,0.0,0.319,1.164,0.114180,14.372926,523.0,563.486792,80.213382


In [9]:
df.isna().sum()

Real time storage           0
Precipitation               0
Precipitation_(-1d)         0
Precipitation_(-7d)         0
Volumetric Soil moisture    0
2m_Temp                     0
Discharge                   0
Evaporation                 0
Sat_Area                    0
dtype: int64

In [10]:
from sklearn.preprocessing import MinMaxScaler

#Normalize data

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [11]:
sequence_length = 16
num_features = len(df.columns)

In [12]:
# Create sequences and corresponding labels
sequences = []
labels = []
for i in range(len(scaled_data) - sequence_length):
    seq = scaled_data[i:i+sequence_length]
    label = scaled_data[i+sequence_length][0]  # '_tempm' column index
    sequences.append(seq)
    labels.append(label)

 # Convert to numpy arrays
sequences = np.array(sequences)
labels = np.array(labels)

# Split into train and test sets
train_size = int(0.8 * len(sequences))
train_x, test_x = sequences[:train_size], sequences[train_size:]
train_y, test_y = labels[:train_size], labels[train_size:]

print("Train X shape:", train_x.shape)
print("Train Y shape:", train_y.shape)
print("Test X shape:", test_x.shape)
print("Test Y shape:", test_y.shape)

Train X shape: (452, 16, 9)
Train Y shape: (452,)
Test X shape: (114, 16, 9)
Test Y shape: (114,)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Create the LSTM model
model = Sequential()

# Add LSTM layers with dropout
model.add(LSTM(units=128, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=32, return_sequences=False))
model.add(Dropout(0.2))

# Add a dense output layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

2024-08-09 15:24:26.015971: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 15:24:26.053086: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-09 15:24:26.053757: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 15:24:39.109454: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 16, 128)           70656     
                                                                 
 dropout (Dropout)           (None, 16, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 16, 64)            49408     
                                                                 
 dropout_1 (Dropout)         (None, 16, 64)            0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 3

In [15]:

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/ML_GW_GIS/best_model_weights.keras', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    train_x, train_y,
    epochs=100,
    batch_size=64,
    validation_split=0.2,  # Use part of the training data as validation
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/100
5/6 [========================>.....] - ETA: 0s - loss: 0.1799

PermissionDeniedError: /content; Permission denied

In [ ]:
import tensorflow as tf
# Evaluate the best model on the test set
best_model = tf.keras.models.load_model('/content/drive/MyDrive/ML_GW_GIS/best_model_weights.keras')
test_loss = best_model.evaluate(test_x, test_y)
print("Test Loss:", test_loss)

In [ ]:

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Assuming you have trained the model and have the 'best_model' object
# Also, 'test_x' and 'test_y' should be available

# Predict temperatures using the trained model
predictions = best_model.predict(test_x)

# Calculate evaluation metrics
mae = mean_absolute_error(test_y, predictions)
mse = mean_squared_error(test_y, predictions)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# y_true values
test_y_copies = np.repeat(test_y.reshape(-1, 1), test_x.shape[-1], axis=-1)
true_tem = scaler.inverse_transform(test_y_copies)
true_temp = true_tem[:,0]

# predicted values
prediction = best_model.predict(test_x)
# Repeat predictions to match the number of features the scaler expects
prediction_copies = np.repeat(prediction, test_x.shape[-1], axis=-1)
predicted_temp = scaler.inverse_transform(prediction_copies)[:,0]

# predicted values
#prediction = best_model.predict(test_x)
#prediction_copies = np.repeat(prediction, 6, axis=-1)
#predicted_temp = scaler.inverse_transform(prediction_copies)[:,0]

In [ ]:
print(true_temp)

In [ ]:
# Plotting predicted and actual groundwater
plt.figure(figsize=(10, 6))
plt.plot(df.index[-114:], true_temp[-114:], label='Actual')
plt.plot(df.index[-114:], predicted_temp[-114:], label='Predicted')
plt.title('Prediction vs Actual')
plt.xlabel('Time')
plt.ylabel('Reservoir storage volume time series (km^3)')
plt.legend()
plt.ylim()
plt.show()